In [1]:
from utils.load import load_everthing
x,y = load_everthing()

In [2]:
import random
def make_split_dict(list_of_keys, test_size, val_size, seed = 1234):
    """Takes a list of keys, and the size of the test and val dataset as percentages"""
    random.seed(seed)
    d = dict()
    test_size = 0.2
    val_size = 0.1
    total = len(list_of_keys)
    all_keys = list_of_keys
    test = int(total*test_size)
    val = int(total*val_size)
    test_keys = random.sample(all_keys, test)
    all_keys = [i for i in all_keys if i not in test_keys]
    val_keys = random.sample(all_keys, val)
    all_keys = [i for i in all_keys if i not in val_keys]
    d["train"] = all_keys
    d["test"] = test_keys
    d["val"] = val_keys
    return d

In [5]:
from collections import defaultdict

def test_train_val_split(x,y,test_size, val_size):
    new_x = defaultdict(lambda: defaultdict(dict))
    #We just sample from CosDist. The reason is that we see no reason to sample independently from the different
    #distance metrics, since the goal is simply to split the data up and compare the different distance metrics equally.
    d = make_split_dict(list(x["CosDist"].keys()), test_size, val_size)
    for distance_metric, x_dict in x.items():
        for iso, feature in x_dict.items():
            if iso in d["train"]:
                new_x["train"][distance_metric][iso] = feature
            elif iso in d["test"]:
                new_x["test"][distance_metric][iso] = feature
            elif iso in d["val"]:
                new_x["val"][distance_metric][iso] = feature
    new_y = defaultdict(dict)
    for iso, label in y.items():
        if iso in d["train"]:
            new_y["train"][iso] = label
        if iso in d["test"]:
            new_y["test"][iso] = label
        if iso in d["val"]:
            new_y["val"][iso] = label
    return new_x, new_y

In [6]:
new_x,new_y = test_train_val_split(x,y,0.2,0.1)